# Display Dublin Office Location on Map

In [1]:
import folium
dublin_office_coordinates = [53.339428, -6.257664]
dublin_office_map = folium.Map(location=dublin_office_coordinates, zoom_start=18)

folium.Marker(
    location=dublin_office_coordinates,
    popup="Dublin Office",
    icon=folium.Icon(color="blue", icon="briefcase")
).add_to(dublin_office_map)

# Dublin Office on Map
### (Click on Marker to view Details)

In [2]:
dublin_office_map

# Fetch Customer Data and Create JSON Object

In [3]:
import urllib.request
try:
  url = "https://gist.githubusercontent.com/shadab16/ebf141802f90a51c42350ca2dd98ec7c/raw/91eec80d5acb4c84db7afc92a9b59c6780b7b81e/customers.txt"
  if (urllib.request.urlopen(url).getcode() != 200):
    raise HTTPError('Check API URL used for fetching data')
  else:
    customers_file_data_byte = urllib.request.urlopen(url).read()
    customers_file_data_string = ('{"customers\":[' + customers_file_data_byte.decode("utf-8") + ']}').replace("\n",",")
except:
  raise HTTPError('Check API URL used for fetching data')

In [4]:
import json
try:
  customer_data = json.loads(customers_file_data_string)
except:
  raise JSONDecodeError()

# Display all Customers in Table Format

In [5]:
import pandas
column_names = {"user_id": "User ID", "name" : "Name", "latitude" : "Latitude", "longitude" : "Longitude"}
print(pandas.DataFrame(customer_data['customers'], columns=['user_id', 'name', 'latitude', 'longitude']
  ).sort_values(by=['user_id']).rename(columns = column_names))

    User ID                Name    Latitude    Longitude
1         1        Alice Cahill    51.92893    -10.27699
2         2         Ian McArdle  51.8856167  -10.4240951
3         3        Jack Enright  52.3191841   -8.5072391
12        4           Ian Kehoe  53.2451022    -6.238335
13        5        Nora Dempsey  53.1302756   -6.2397222
9         6     Theresa Enright  53.1229599   -6.2705202
5         7         Frank Kehoe  53.4692815    -9.436036
6         8         Eoin Ahearn  54.0894797     -6.18671
10        9        Jack Dempsey  52.2559432   -7.1048927
11       10  Georgina Gallagher   52.240382    -6.972413
14       11    Richard Finnegan   53.008769   -6.1056711
0        12   Christina McArdle   52.986375    -6.043701
16       13        Olive Ahearn          53           -7
17       14        Helen Cahill   51.999447    -9.742744
18       15      Michael Ahearn      52.966       -6.463
19       16          Ian Larkin   52.366037    -8.179118
20       17     Patricia Cahill

# Display all Customers on Map

In [6]:
customer_map = folium.Map(location=dublin_office_coordinates, zoom_start=7)
latitudes = []
longitudes = []
user_details = []
for customer in customer_data['customers']:
  latitudes.append(float(customer['latitude']))
  longitudes.append(float(customer['longitude']))
  user_details.append(str(customer['user_id']) + ' ' + customer['name'])
for index in range (0, len(latitudes)-1):
  coordinates = [latitudes[index], longitudes[index]]
  user_detail = user_details[index]
  folium.Marker(location = coordinates, popup = user_detail, icon = folium.Icon(color = 'red', icon='user')).add_to(customer_map)

# All Customers on Map
### (Click on Markers to view User Details)

In [7]:
customer_map

# Compute Distance from Dublin Office using Great Circle Distance Formula

In [8]:
from math import radians, degrees, sin, cos, asin, acos, sqrt
def distance_from_dublin_office(location_latitude, location_longitude):
  if ((type(location_latitude) == type(True)) or (type(location_longitude) == type(True))):
    raise TypeError('Coordinates should be valid int or float values')
  else:
    try:
      location_latitude = float(location_latitude)
      location_longitude = float(location_longitude)
      dublin_office_latitude = 53.339428
      dublin_office_longitude = -6.257664
      dublin_office_latitude, dublin_office_longitude, location_latitude, location_longitude = map(radians, 
        [dublin_office_latitude, dublin_office_longitude, location_latitude, location_longitude])
      return 6371.0088 * (acos(sin(dublin_office_latitude) * sin(location_latitude) + cos(dublin_office_latitude) * 
        cos(location_latitude) * cos(dublin_office_longitude - location_longitude)))
    except:
      raise TypeError('Coordinates should be valid int or float values')

# Add Distance Data to Customer Data JSON Object

In [9]:
for customer in customer_data['customers']:
  customer.update({"distance" : distance_from_dublin_office(float(customer['latitude']), float(customer['longitude']))})

# Filter Invited Customers within 100km Range from Dublin Office

In [10]:
invited_cutomers = []
for customer in customer_data['customers']:
   if customer['distance'] <= 100:
     invited_cutomers.append(customer)
def get_user_id(invited_cutomers):
  return invited_cutomers.get('user_id')
invited_cutomers.sort(key = get_user_id)

# Display Invited Customers in Table Format

In [11]:
import pandas
column_names = {"user_id": "User ID", "name" : "Name", "distance" : "Distance"}
print(pandas.DataFrame(invited_cutomers, columns=["user_id", "name", "distance"]).rename(columns = column_names))

    User ID               Name   Distance
0         4          Ian Kehoe  10.566951
1         5       Nora Dempsey  23.287353
2         6    Theresa Enright  24.085393
3         8        Eoin Ahearn  83.532647
4        11   Richard Finnegan  38.137621
5        12  Christina McArdle  41.768783
6        13       Olive Ahearn  62.231788
7        15     Michael Ahearn  43.722548
8        17    Patricia Cahill  96.078732
9        23     Eoin Gallagher  82.695040
10       24       Rose Enright  89.031157
11       26    Stephen McArdle  98.874736
12       29      Oliver Ahearn  72.201885
13       30       Nick Enright  82.642964
14       31         Alan Behan  44.290884
15       39        Lisa Ahearn  38.358068


# Display Invited Customers on Map

In [12]:
invited_customer_map = folium.Map(location=dublin_office_coordinates, zoom_start=7)
latitudes = []
longitudes = []
user_details = []
for customer in invited_cutomers:
  latitudes.append(float(customer.get('latitude')))
  longitudes.append(float(customer.get('longitude')))
  user_details.append(str(customer.get('user_id')) + ' ' + customer.get('name'))
for index in range (0, len(latitudes)-1):
  coordinates = [latitudes[index], longitudes[index]]
  user_detail = user_details[index]
  folium.Marker(location = coordinates, popup = user_detail, icon = folium.Icon(color = 'blue', icon='user')).add_to(invited_customer_map)

# Invited Customers on Map
### (Click on Markers to view User Details)

In [13]:
invited_customer_map

# Unit Testing

In [14]:
import unittest
class TestProdApp(unittest.TestCase):
  # ------------------------------------- Testing Data Type of Inputs -------------------------------------
  def test_my_distance(self):
    self.assertEqual(distance_from_dublin_office(19.075800,72.910011), 7605.095952330754)
  def test_not_my_distance(self):
    self.assertNotEqual(distance_from_dublin_office(19.075800,72.910011), 7605)
  def test_type_error_for_string_float(self):
    self.assertRaises(TypeError, distance_from_dublin_office, 'hello', 5.0)
  def test_type_error_for_string_int(self):
    self.assertRaises(TypeError, distance_from_dublin_office, 'hello', 5)
  def test_type_error_for_bool_float(self):
    self.assertRaises(TypeError, distance_from_dublin_office, True, 5.0)
  def test_type_error_for_bool_int(self):
    self.assertRaises(TypeError, distance_from_dublin_office, True, 5)
  def test_type_error_for_complex_float(self):
    self.assertRaises(TypeError, distance_from_dublin_office, 1 + 2j, 5.0)
  def test_type_error_for_complex_int(self):
    self.assertRaises(TypeError, distance_from_dublin_office, 1 + 2j, 5)
  # --------------------------------------- Testing Invited Guests ---------------------------------------
  def test_invited_guests(self):
    for customer in invited_cutomers:
      self.assertLessEqual(distance_from_dublin_office(customer['latitude'],customer['longitude']), 100,
        msg="Customers living beyond the range of 100kms are being invited")
  # -------------------------------------- Testing Uninvited Guests --------------------------------------
  def test_uninvited_guests(self):
    for customer in customer_data['customers']:
      if customer not in invited_cutomers:
        self.assertGreater(distance_from_dublin_office(customer['latitude'],customer['longitude']), 100,
          msg="Customers living in the range of 100kms are not being invited")
  # --------------------------------------- Testing API Integration ---------------------------------------
  def test_if_api_is_correct(self):
    self.assertTrue(urllib.request.urlopen(url).getcode() == 200)
  def test_if_api_is_not_incorrect(self):
    self.assertFalse(urllib.request.urlopen(url).getcode() != 200)
  # ------------------------------------------ Testing JSON Data ------------------------------------------
  def test_user_id_key_in_json(self):
    self.assertTrue('user_id' in customer in customer_data['customers'])
  def test_name_key_in_json(self):
    self.assertTrue('name' in customer in customer_data['customers'])
  def test_latitude_key_in_json(self):
    self.assertTrue('latitude' in customer in customer_data['customers'])
  def test_longitude_key_in_json(self):
    self.assertTrue('longitude' in customer in customer_data['customers'])

unittest.main(argv=[''], exit=False)

................
----------------------------------------------------------------------
Ran 16 tests in 0.071s

OK
